https://colab.research.google.com/github/TavernAI/TavernAI/blob/main/colab/GPU.ipynb<br>

Works with:<br>
KoboldAI https://github.com/KoboldAI/KoboldAI-Client<br>
Pygmalion https://huggingface.co/PygmalionAI/<br>
<br>
**Links**<br>
TavernAI Github https://github.com/TavernAI/TavernAI<br>
TavernAI Discord https://discord.gg/zmK2gmr45t<br>
TavernAI Boosty https://boosty.to/tavernai
<pre>
 Tavern.AI/ \ /  ^   ^ ^ ^    ~~~~ ^ \     /  ^ ^   ^ ^/ ^  ^ \/^  ^    \
         /^ ^\ ^ ^^ ^^  ^ ^  ~~   ^  ^\   /  ^  ^ ^ ^ / ^ ^  ^/   ^ ^    \
        /^ ^ ^\^  ^^ ^ ^   _||____   ^ \ /  ^  ^ ^   /       /  ^  ^  ^   \
 /\ /\ /\   ^  \  /\ /\   /\\\\\\\\   ^ \  ^ /\ /\ /\   /\ /\ /\  ^ ^  ^/\
//\\/\\/\\   ^  \//\\/\\ /__\\\\\\\\  _, \  //\\/\\/\\ //\\/\\/\\  ^ ^ //\\
//\\/\\/\\       //\\/\\ |__|_|_|__|   \__, //\\/\\/\\ //\\/\\/\\     ///\\\
 || || (@＾◡＾)(≖ ‸ ≖*) ( ←_← )\| /|   /\ \ヽ(°ㅂ°╬) |( Ψ▼ｰ▼)∈ (O_O; )  |||
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~  ~~~~~ ~~ 
</pre>
**Launch Instructions**<br>
1. Click the launch button.
2. Wait for the environment and model to load
3. After initialization, a TavernAI link will appear

In [ ]:
#@title <b>TavernAI</b>
#@markdown <- Click For Start (≖ ‸ ≖ ✿)

Model = "Pygmalion 6B" #@param [ "Pygmalion 6B", "Pygmalion 6B Dev"] {allow-input: true}
Version = "Official" 
Provider = "Cloudflare" 
use_google_drive = False

!nvidia-smi
import subprocess
import time
import sys
import os
from google.colab import drive
if use_google_drive:
  drive.mount('/content/drive/')
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

Revision = ""

if Model == "Pygmalion 6B":
  Model = "PygmalionAI/pygmalion-6b"
  path = ""
  download = ""
  Version = "United"
elif Model == "Pygmalion 6B Dev":
  Model = "PygmalionAI/pygmalion-6b"
  Revision = "--revision dev"
  path = ""
  Version = "United"
  download = ""

if Provider == "Localtunnel":
  tunnel = "--localtunnel yes"
else:
  tunnel = ""

#Henk's KoboldAI script
!wget https://koboldai.org/ckds && chmod +x ckds
!./ckds --init only
p = subprocess.Popen(['/content/ckds', '--model', Model], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

#Tricks
url = ''
while True:
    line = p.stdout.readline().decode().strip()
    if "KoboldAI has finished loading and is available at the following link: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link: ")[1]
        print(url)
        break
    if "KoboldAI has finished loading and is available at the following link for UI 1: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link for UI 1: ")[1]
        print(url)
        break
    if not line:
        break
    print(line)
    if "INIT" in line and "Transformers" in line:
      print("Model loading... (It will take 2 - 5 minutes)")

#TavernAI
%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v
!git clone https://github.com/TavernAI/taicolabtest
%cd taicolabtest
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
!nohup node server.js &
time.sleep(3)
print('KoboldAI LINK:')
print(url)
print('')
print('###TavernAI LINK###')
!lt --port 8000
